# SQLAlchemy 1.4 / 2.0 Tutorial

## Establishing Connectivity - the Engine

In [14]:
import os.path
import yaml
from sqlalchemy import create_engine, text

config_path = os.path.expanduser('~\\.sqlalchemy\\config.yaml')

with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

db_path = (
    f"postgresql://{config['username']}:{config['password']}"
    f"@{config['host']}:{config['port']}/{config['db_name']}"
)
engine = create_engine(db_path, echo=True, future=True)

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())


2021-11-18 13:58:48,954 INFO sqlalchemy.engine.Engine select version()
2021-11-18 13:58:48,954 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,955 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-18 13:58:48,956 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,957 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-11-18 13:58:48,958 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,959 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 13:58:48,960 INFO sqlalchemy.engine.Engine select 'hello world'
2021-11-18 13:58:48,961 INFO sqlalchemy.engine.Engine [generated in 0.00126s] {}
[('hello world',)]
2021-11-18 13:58:48,963 INFO sqlalchemy.engine.Engine ROLLBACK


## Working with Transactions and the DBAPI

### Committing Changes

In [15]:
# commit as you go
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE FUNC( x integer, y integer )"))
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x": 1, "y": 1}, {"x": 2, "y": 4}])
    conn.commit()

# begin once
with engine.begin() as conn:
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x": 6, "y": 8}, {"x": 9, "y": 10}])


2021-11-18 15:15:22,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:15:22,627 INFO sqlalchemy.engine.Engine CREATE TABLE FUNC( x integer, y integer )
2021-11-18 15:15:22,627 INFO sqlalchemy.engine.Engine [generated in 0.00121s] {}
2021-11-18 15:15:22,652 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:15:22,653 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ({'x': 1, 'y': 1}, {'x': 2, 'y': 4})
2021-11-18 15:15:22,655 INFO sqlalchemy.engine.Engine COMMIT
2021-11-18 15:15:22,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:15:22,659 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:15:22,659 INFO sqlalchemy.engine.Engine [cached since 0.007468s ago] ({'x': 6, 'y': 8}, {'x': 9, 'y': 10})
2021-11-18 15:15:22,663 INFO sqlalchemy.engine.Engine COMMIT


### Basics of Statement Execution

In [17]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for dict_row in result.mappings():
        x = dict_row['x']
        y = dict_row['y']
        print(x, y)

    result = conn.execute(text("SELECT x,y FROM FUNC where y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    conn.execute(text("INSERT INTO FUNC(x,y) VALUES (:x, :y)"),
                 [{"x": 11, "y": 12}, {"x": 13, "y": 14}])
    conn.commit()

    # bundling parameters
    stmt = text(
        "SELECT x, y FROM FUNC WHERE y > :y ORDER BY x,y"
        ).bindparams(y=6)
    result = conn.execute(stmt)
    for row in result:
        print(f"x: {row.x} y: {row.y}")


2021-11-18 15:56:35,409 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:56:35,410 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-18 15:56:35,410 INFO sqlalchemy.engine.Engine [generated in 0.00166s] {}
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2021-11-18 15:56:35,412 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-18 15:56:35,413 INFO sqlalchemy.engine.Engine [cached since 0.004293s ago] {}
1 1
2 4
6 8
9 10
2021-11-18 15:56:35,415 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC where y > %(y)s
2021-11-18 15:56:35,415 INFO sqlalchemy.engine.Engine [generated in 0.00051s] {'y': 2}
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2021-11-18 15:56:35,420 INFO sqlalchemy.engine.Engine INSERT INTO FUNC(x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:56:35,421 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ({'x': 11, 'y': 12}, {'x': 13, 'y': 14})
2021-11-18 15:56:35,422 INFO sqlalchemy.engine.Engine COMMIT
2021-11-18 15:56:35,423 INFO sqlalchemy.engine.Engine BEGIN (impl

### Executing with an ORM Session

In [18]:
from sqlalchemy.orm import Session
stmt = text(
    "SELECT x, y FROM FUNC WHERE y > :y ORDER BY x, y"
).bindparams(y=6)

with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    result = session.execute(
        text("UPDATE FUNC SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}]
    )
    session.commit()  # To save, you should commit


2021-11-18 16:02:08,297 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine SELECT x, y FROM FUNC WHERE y > %(y)s ORDER BY x, y
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine [generated in 0.00039s] {'y': 6}
x: 6 y: 8
x: 9 y: 10
x: 11 y: 12
x: 13 y: 14
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine UPDATE FUNC SET y=%(y)s WHERE x=%(x)s
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine [generated in 0.00036s] ({'y': 11, 'x': 9}, {'y': 15, 'x': 13})
2021-11-18 16:02:08,319 INFO sqlalchemy.engine.Engine COMMIT


## Working with Database Metadata

### Setting up Metadata with Table objects

In [31]:
from sqlalchemy import MetaData
metadata_obj = MetaData()  # a set of Table object

print(metadata_obj)

MetaData()


In [32]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",  # table name
    metadata_obj,  # 'metadata_obj' has this table.
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)


### Declaring Simple Constraints

In [33]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [34]:
from sqlalchemy import ForeignKey
#  ForeignKey : we can omit the datatype for this column
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('emain_address', String, nullable=False)
)


### Emitting DDL to the Database

In [37]:
# create
metadata_obj.create_all(engine)

2021-11-18 16:22:27,183 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 16:22:27,184 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-18 16:22:27,185 INFO sqlalchemy.engine.Engine [cached since 20.8s ago] {'name': 'user_account'}
2021-11-18 16:22:27,187 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-18 16:22:27,188 INFO sqlalchemy.engine.Engine [cached since 20.8s ago] {'name': 'address'}
2021-11-18 16:22:27,189 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id SERIAL NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2021-11-18 16:22:27,189 INFO sqlalchemy.engine.Engine [no key 0.00050s] {}
2021-11-18 16:22:27,204 INFO sqlalchemy.engine.Engine 
CREATE TABLE address (
	id SERIAL NOT NUL

In [36]:
# delete
metadata_obj.drop_all(engine)

2021-11-18 16:22:16,961 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 16:22:16,962 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-18 16:22:16,963 INFO sqlalchemy.engine.Engine [cached since 10.58s ago] {'name': 'user_account'}
2021-11-18 16:22:16,968 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-18 16:22:16,968 INFO sqlalchemy.engine.Engine [cached since 10.58s ago] {'name': 'address'}
2021-11-18 16:22:16,969 INFO sqlalchemy.engine.Engine 
DROP TABLE address
2021-11-18 16:22:16,971 INFO sqlalchemy.engine.Engine [no key 0.00182s] {}
2021-11-18 16:22:16,976 INFO sqlalchemy.engine.Engine 
DROP TABLE user_account
2021-11-18 16:22:16,977 INFO sqlalchemy.engine.Engine [no key 0.00057s] {}
2021-11-18 16:22:16,978 INFO sqla

### Defining Table Metadata with the ORM

In [40]:
# Setting up the Registry
from sqlalchemy.orm import registry
mapper_registry = registry()
mapper_registry.metadata
Base = mapper_registry.generate_base()  # declarative base


In [41]:
# Declaring Mapped Classes (using base)
from sqlalchemy.orm import relationship


class User(Base):
    __tablename__ = 'user_account'

    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")
    # back_populates: Establishing a relationship in both direction.

    def __repr__(self):
        return (
            f"User(id={self.id!r}, name={self.name!r}, "
            f"fullname={self.fullname!r})"
        )
        # !r : repr(). a formal string that the interpreter can recognize


class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"


In [42]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [43]:
# Other Mapped Class Details
sandy = User(name="sandy", fullname="Sandy Cheeks")  # create a instance
sandy  # __repr__() is used

User(id=None, name='sandy', fullname='Sandy Cheeks')

In [44]:
# Emitting DDL to the database. (three ways to emit DDL)
# 1) MetaData.create_all(engine)
# 2) mapper_registry.metadata.create_all(engine)
# 3) Base.metadata.create_all(engine)


In [46]:
# Combining Core Table Declarations with ORM Declarative
# class User(Base):
#    __table__ = user_table  # we don't have to write column infos.

### Table Reflection

In [54]:
# Load table from existing database as ORM object
# (not declaring table directly)
funcrefl = Table("func", metadata_obj, autoload_with=engine)


In [55]:
funcrefl

Table('func', MetaData(), Column('x', INTEGER(), table=<func>), Column('y', INTEGER(), table=<func>), schema=None)